In [6]:
import numpy as np

# load dialogues_and_speakers.npy
data = np.load('../Book Preprocessing/dialogues_and_speakers.npy')

for dialogue, speaker in data:
    print('Speaker:', speaker)
    print('Dialogue:', dialogue)

# create a list of the first 25 lines of dialogue
test_dialogue_lines = []

# give me only the first 25 lines
for dialogue, speaker in data[:25]:
    test_dialogue_lines.append((dialogue, speaker))

Speaker: المعلق
Dialogue: مجنون ليلى
Speaker: المعلق
Dialogue: الفصل الأول
Speaker: المعلق
Dialogue: (ساحة أمام خيام المهدي في حي بني عامر - مجلس من مجالس السمر في هذه
            الساحة - فتية وفتيات من الحي يسمرون في أوائل الليل، وفي أيدي الفتيات صوف ومغازل يلهون
            بها وهم يتحدثون — تخرج ليلى من خيام أبيها عند ارتفاع الستار ويدها في يد ابن ذريح)
Speaker: المعلق
Dialogue: ليلى
Speaker: ليلى
Dialogue: دعي الغزْلَ سلمى وحَيِّي معي
Speaker: ليلى
Dialogue: منارَ الحِجَازِ فَتَى يَثْرِبِ
Speaker: المعلق
Dialogue: (تصافحه سلمى)
Speaker: ليلى
Dialogue: ويا هِنْدُ هذا أديبُ الحِجازِ
Speaker: ليلى
Dialogue: هلمِّي بمَقْدَمِهِ رَحِّبِي
Speaker: المعلق
Dialogue: (تصافحه هند ويحتفي به السامرون)
Speaker: المعلق
Dialogue: سعد
Speaker: سعد
Dialogue: أمن يثربٍ أنت آتٍ؟
Speaker: المعلق
Dialogue: ابن ذريح
Speaker: ابن ذريح
Dialogue: أجل
Speaker: ابن ذريح
Dialogue: من البلدِ القُدُس الطيِّب
Speaker: المعلق
Dialogue: ليلى
Speaker: ليلى
Dialogue: أيابنَ ذَريحٍ لقينا الغمام
Speaker: المعلق
Dialog

In [2]:
characters = set()
for dialogue, speaker in test_dialogue_lines:
    characters.add(speaker)

print('Characters:', characters)


Characters: {'منازل', 'هند', 'المعلق', 'قيس', 'عبلة', 'سعد', 'عفراء', 'زياد', 'ليلى', 'أصوات', 'ابن ذريح', 'المهدي', 'فتاة', 'سلمى', 'بشر'}


In [3]:
# print characters not in character_id dict

character_id = {
    "ليلى": "XB0fDUnXU5powFXDhCwa",
    "المعلق": "nPczCjzI2devNBz1zQrb",
    "سعد": "iP95p4xoKVk53GoZ742B",
    "هند": "Xb7hH8MSUJpSbSDYk0k2",
    "عبلة": "Xb7hH8MSUJpSbSDYk0k2",
    "ابن ذريح": "N2lVS1w4EtoT3dr4eOWO",
}


for character in characters:
    if character not in character_id:
        print('Character not in character_id:', character)

Character not in character_id: منازل
Character not in character_id: قيس
Character not in character_id: عفراء
Character not in character_id: زياد
Character not in character_id: أصوات
Character not in character_id: المهدي
Character not in character_id: فتاة
Character not in character_id: سلمى
Character not in character_id: بشر


In [ ]:
import requests
import json

# Load API keys from a JSON file
with open('api_keys.json') as file:
    api_keys = json.load(file)

# Convert the API keys dictionary to a list of keys
api_key_list = list(api_keys.values())

# Function to update the API key in headers
def update_api_key(headers, new_key):
    headers["xi-api-key"] = new_key
    return headers

character_id = {
    "ليلى": "XB0fDUnXU5powFXDhCwa",
    "المعلق": "nPczCjzI2devNBz1zQrb",
    "سعد": "iP95p4xoKVk53GoZ742B",
    "هند": "Xb7hH8MSUJpSbSDYk0k2",
    "عبلة": "Xb7hH8MSUJpSbSDYk0k2",
    "ابن ذريح": "N2lVS1w4EtoT3dr4eOWO",
}

headers = {
    "xi-api-key": api_key_list[0],  # Start with the first API key
    "Content-Type": "application/json"
}

for audioNum, (dialogue, speaker) in enumerate(test_dialogue_lines):
    payload = {
        "model_id": "eleven_multilingual_v2",
        "text": f"{dialogue}"
    }

    url = f"https://api.elevenlabs.io/v1/text-to-speech/{character_id[speaker]}"

    for api_key in api_key_list:
        response = requests.post(url=url, json=payload, headers=update_api_key(headers, api_key))

        if response.status_code == 200:
            with open(f"testAudios-first20lines/{audioNum+1}-{speaker}.mp3", 'wb') as audio_file:
                audio_file.write(response.content)
            print(f"Audio file {audioNum+1} saved successfully.")
            break  # Exit the API key loop on success
        else:
            print(f"Retry with next API key due to status code:", response.status_code)
    else:
        # This part is executed only if the loop was not broken
        print(f"Failed to retrieve audio {audioNum+1}. Tried all API keys.")
        break  # Exit the dialogue loop since all keys failed

In [19]:
# combine all the audio files into one
from moviepy.editor import concatenate_audioclips, AudioClip, AudioFileClip

audio_clips = []

for audioNum, (dialogue, speaker) in enumerate(test_dialogue_lines):
    audio_clips.append(AudioFileClip(f"../Book Preprocessing/testAudios-first20lines/{audioNum+1}-{speaker}.mp3"))

# Function to create a silent audio clip of a specified duration
def make_silent_clip(duration):
    # Creating a silent clip. This function generates silence.
    return AudioClip(lambda t: np.zeros(1), duration=duration, fps=44100)

# Duration of the pause in seconds
pause_duration = 0.25

# Create silent clips for pauses
silent_clip = make_silent_clip(pause_duration)

# Insert silent clips between the audio clips
audio_clips_with_pause = [clip for pair in zip(audio_clips, [silent_clip] * (len(audio_clips) - 1)) for clip in pair] + [audio_clips[-1]]

# Concatenate all clips including the silent ones
final_audio = concatenate_audioclips(audio_clips_with_pause)

final_audio.write_audiofile("chapter1.mp3")
print("Combined audio file saved successfully.")

MoviePy - Writing audio in combined_audio.mp3


MoviePy - Done.
Combined audio file saved successfully.
